## Region of interest

Path to geojson files of cities, Toulouse, Cusco, and Calama

In [ ]:
path_to_roi_geojson = r'toulouse.geojson'

Load ROI from geojson file

In [ ]:
import json
f = open(path_to_roi_geojson)
dat = json.load(f)
roi = dat['coordinates']

import ee
ee.Initialize()
ee_roi = ee.Geometry.Polygon(roi)

Visualize the ROI on a base map

In [ ]:
import geemap
Map = geemap.Map()
Map.centerObject(ee_roi,13)
Map.addLayer(ee_roi, {'color': 'FF0000'})
Map

## Retrive S2 images suit the criterion

In [ ]:
import ee
import os
def clipImg(x):
    return x.clip(ee_roi)

s2_bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']
collection = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(ee_roi) \
    .filterDate('2018-02-01', '2018-02-15') \
    .map(clipImg) \
    .select(s2_bands)

print(collection.aggregate_array('system:index').getInfo())
img = collection.first()

## Generate cloud free data

In [ ]:
import sys
#sys.path.append(r'C:\Users\Jingliang\Documents\projects\Sentinel-1_time_series\Extra_cities\src')
import sentinel_2_cloud_free

# This function solve everything, but the data can not be downloaded
img = sentinel_2_cloud_free.exportCloudFreeSen2(ee_roi, '2018-01-01', '2018-02-01')

# img = sentinel_2_cloud_free.mergeCollection(collection)
# img = collection.first()
Map.addLayer(img, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 4000})
Map

## Data download

In [ ]:
save_dir = r'to-be-set'
# geemap.ee_export_image_collection(collection, out_dir=save_dir, region=ee_roi, scale=10)
geemap.ee_export_image(img, filename='s2_toulouse_feb.tif', scale=10, region=ee_roi, file_per_band=True)
